# Section 2 : Downloading data

In [ ]:
import wrds
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

## Getting hands on the right library and table

In [2]:
db=wrds.Connection(wrds_username='arthurdhonneur')

Loading library list...
Done


In [3]:
db.list_libraries()

['aha_sample',
 'ahasamp',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'bank_premium_samp',
 'banksamp',
 'block',
 'block_all',
 'boardex_trial',
 'boardsmp',
 'bvd_amadeus_trial',
 'bvd_bvdbankf_trial',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'candid_samp',
 'cboe',
 'cboe_all',
 'cboe_sample',
 'cboesamp',
 'cddsamp',
 'ciqsamp',
 'ciqsamp_capstrct',
 'ciqsamp_common',
 'ciqsamp_keydev',
 'ciqsamp_pplintel',
 'ciqsamp_ratings',
 'ciqsamp_transactions',
 'ciqsamp_transcripts',
 'cisdmsmp',
 'columnar',
 'comp',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global_daily',
 'comp_na_daily_all',
 'comp_segments_hist_daily',
 'compsamp',
 'compsamp_all',
 'compsamp_snapshot',
 'compseg',
 'contrib',
 'contrib_as_filed_financials',
 'contrib_ceo_turnover',
 'contrib_char_returns',
 'contrib_corporate_culture',
 'contrib_general',
 'contrib_global_factor',
 'contrib_intangible_value',
 'contrib_kpss',
 'contrib_liva',
 'crsp',
 'crsp_a_ccm',
 'crsp_a_inde

In [4]:
db.list_tables(library='wrdsapps_windices')


['dwcountryreturns', 'mwcountryreturns', 'wcountryconstituents']

In [5]:
df = db.get_table(library='wrdsapps_windices', table='mwcountryreturns')

df.head()

,fic,date,country,currency,mportret,mportretx
0,AUS,1986-07-31,AUSTRALIA,AUD,-0.0547,-0.056155
1,AUS,1986-08-29,AUSTRALIA,AUD,0.040118,0.0393
2,AUS,1986-09-30,AUSTRALIA,AUD,0.047842,0.045157
3,AUS,1986-10-31,AUSTRALIA,AUD,0.106395,0.097698
4,AUS,1986-11-28,AUSTRALIA,AUD,0.005453,0.00311


In [11]:
df.to_csv('./data/wrds/country_index.csv', index=False)

In [12]:
#---------------------------------------------
# Value Weighted Index Returns
#---------------------------------------------
Rm=db.raw_sql("""select  date,vwretd from crsp.msi 
                where date>='2002-01-01' and date<='2024-12-31'
                """,date_cols=['date'])
Rm = Rm.rename(columns={'vwretd':'Rm'})

Rm.head()

,date,Rm
0,2002-01-31,-0.015966
1,2002-02-28,-0.0217
2,2002-03-28,0.044698
3,2002-04-30,-0.0496
4,2002-05-31,-0.01051


In [13]:
Rm.to_csv('./data/wrds/us.csv', index=False)

In [14]:
#---------------------------------------------
# Risk Free Rate 
#---------------------------------------------
Rf=db.raw_sql("""select  mcaldt,tmytm 
           from crsp.tfz_mth_rf            
            where kytreasnox = 2000001 
           and mcaldt>='2002-01-01'
            and mcaldt<='2024-12-31'""", date_cols=['mcaldt'])
Rf['tmytm']=Rf['tmytm']/12/100
Rf=Rf.rename(columns={ "mcaldt": "date","tmytm": "rf"})

Rf.tail()

,date,rf
271,2024-08-30,0.00438
272,2024-09-30,0.003826
273,2024-10-31,0.003752
274,2024-11-29,0.003475
275,2024-12-31,0.003337


In [15]:
Rf.to_csv('./data/wrds/risk_free_rate.csv', index=False)

## Looking at the fx data


In [ ]:
import glob

paths_fx = glob.glob('data/fx/*.csv')

for path in paths_fx:
    curr = path.split('/')[-1].split('.')[0]
    print(f'the currency :{curr}')
    
    df_tamp = pd.read_csv(path)
    print(f'the len of the data :{len(df_tamp)}')
    print('--'*20)

## Looking at the interbank data

In [63]:
paths_rates = glob.glob('data/interbank_rates/*.csv')

mapping_interbank = {}

for path in paths_rates:
    curr = path.split('/')[-1].split('.')[0]
    print(f'the currency :{curr}')
    
    df_tamp = pd.read_csv(path)
    print(f'the len of the data :{len(df_tamp)}')

    mapping_interbank[df_tamp.columns[-1]] = curr
    print('--'*20)

the currency :euro_interbank
the len of the data :276
----------------------------------------
the currency :ch_interbank
the len of the data :276
----------------------------------------
the currency :us_interbank
the len of the data :276
----------------------------------------
the currency :australia_interbank
the len of the data :276
----------------------------------------
the currency :japan_interbank
the len of the data :273
----------------------------------------
the currency :uk_interbank
the len of the data :276
----------------------------------------


In [64]:
mapping_interbank

{'IR3TIB01EZM156N': 'euro_interbank',
 'IR3TIB01CHM156N': 'ch_interbank',
 'IR3TIB01USM156N': 'us_interbank',
 'IR3TIB01AUM156N': 'australia_interbank',
 'IR3TIB01JPM156N': 'japan_interbank',
 'IR3TIB01GBM156N': 'uk_interbank'}

In [61]:
df_tamp.head()

,observation_date,IR3TIB01GBM156N
0,2002-01-01,4.04677
1,2002-02-01,4.05268
2,2002-03-01,4.13055
3,2002-04-01,4.17319
4,2002-05-01,4.15370


## Working on market data

In [34]:
df = pd.read_csv('./data/wrds/country_index.csv')

df = df[(df['date'] >= '2002-01-01') & (df['date'] <= '2024-12-31')]

list_countries = ['AUS', 'FRA', 'DEU', 'JPN', 'CHE','GBR']
df = df[df['fic'].isin(list_countries)]

df.sample(10)

,fic,date,country,currency,mportret,mportretx
4432,FRA,2002-04-30,FRANCE,EUR,-0.043988,-0.045755
5155,GBR,2023-11-30,UNITED KINGDOM,GBP,0.028592,0.023764
1881,CHE,2023-08-31,SWITZERLAND,CHF,-0.024224,-0.024516
8037,JPN,2016-10-31,JAPAN,JPY,0.049256,0.049180
284,AUS,2010-03-31,AUSTRALIA,AUD,0.055007,0.048610
5075,GBR,2017-03-31,UNITED KINGDOM,GBP,0.013045,0.012066
5017,GBR,2012-05-31,UNITED KINGDOM,GBP,-0.062313,-0.067393
1767,CHE,2014-02-28,SWITZERLAND,CHF,0.034613,0.028486
394,AUS,2019-05-31,AUSTRALIA,AUD,0.018947,0.012484
2919,DEU,2010-02-26,GERMANY,EUR,-0.005374,-0.005434


In [35]:
# Get the list of unique countries
countries = df['country'].unique()

# List to hold individual country DataFrames
country_dfs = []

for country in countries:

    print(f'Processing country: {country}')
    # Filter for the specific country
    df_country = df[df['country'] == country].copy()
    
    print(f'Number of rows for {country}: {len(df_country)}')
    
    # Drop the 'country' column as it's now implicit
    df_country = df_country.drop(columns=['country'])
    df_country['date'] = Rm['date'].values
    
    # Rename columns to include country suffix, except 'date'
    df_country = df_country.rename(columns={col: f"{col}_{country}" for col in df_country.columns if col != 'date'})
    
    # Append to list
    country_dfs.append(df_country)

# Merge all country-specific DataFrames on 'date'
from functools import reduce
df_merged = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), country_dfs)

# Optional: sort by date
df_merged = df_merged.sort_values(by='date')

# Display the result
print(df_merged.head())


Processing country: AUSTRALIA
Number of rows for AUSTRALIA: 276
Processing country: SWITZERLAND
Number of rows for SWITZERLAND: 276
Processing country: GERMANY
Number of rows for GERMANY: 276
Processing country: FRANCE
Number of rows for FRANCE: 276
Processing country: UNITED KINGDOM
Number of rows for UNITED KINGDOM: 276
Processing country: JAPAN
Number of rows for JAPAN: 276
  fic_AUSTRALIA       date currency_AUSTRALIA  mportret_AUSTRALIA  \
0           AUS 2002-01-31                AUD            0.027358   
1           AUS 2002-02-28                AUD           -0.001832   
2           AUS 2002-03-28                AUD            0.005199   
3           AUS 2002-04-30                AUD           -0.014302   
4           AUS 2002-05-31                AUD           -0.000204   

   mportretx_AUSTRALIA fic_SWITZERLAND currency_SWITZERLAND  \
0             0.027313             CHE                  CHF   
1            -0.004358             CHE                  CHF   
2            -0.

In [36]:
pd.set_option('display.max_columns', None)

df_merged.head()

,fic_AUSTRALIA,date,currency_AUSTRALIA,mportret_AUSTRALIA,mportretx_AUSTRALIA,fic_SWITZERLAND,currency_SWITZERLAND,mportret_SWITZERLAND,mportretx_SWITZERLAND,fic_GERMANY,currency_GERMANY,mportret_GERMANY,mportretx_GERMANY,fic_FRANCE,currency_FRANCE,mportret_FRANCE,mportretx_FRANCE,fic_UNITED KINGDOM,currency_UNITED KINGDOM,mportret_UNITED KINGDOM,mportretx_UNITED KINGDOM,fic_JAPAN,currency_JAPAN,mportret_JAPAN,mportretx_JAPAN
0,AUS,2002-01-31,AUD,0.027358,0.027313,CHE,CHF,-0.022126,-0.022146,DEU,EUR,0.012277,0.011491,FRA,EUR,-0.027105,-0.027306,GBR,GBP,-0.010866,-0.011720,JPN,JPY,-0.056655,-0.056710
1,AUS,2002-02-28,AUD,-0.001832,-0.004358,CHE,CHF,0.011661,0.011646,DEU,EUR,-0.008807,-0.008816,FRA,EUR,-0.004394,-0.004447,GBR,GBP,-0.003341,-0.007072,JPN,JPY,0.042256,0.041998
2,AUS,2002-03-28,AUD,0.005199,-0.001914,CHE,CHF,0.048759,0.045570,DEU,EUR,0.049328,0.048470,FRA,EUR,0.052081,0.051943,GBR,GBP,0.038930,0.033661,JPN,JPY,0.053119,0.048294
3,AUS,2002-04-30,AUD,-0.014302,-0.015424,CHE,CHF,-0.004109,-0.007961,DEU,EUR,-0.045031,-0.047093,FRA,EUR,-0.043988,-0.045755,GBR,GBP,-0.013362,-0.015411,JPN,JPY,0.020586,0.020574
4,AUS,2002-05-31,AUD,-0.000204,-0.001752,CHE,CHF,0.002963,-0.000639,DEU,EUR,-0.025703,-0.033068,FRA,EUR,-0.028016,-0.035487,GBR,GBP,-0.011482,-0.013937,JPN,JPY,0.035987,0.035920


In [37]:
df_merged = df_merged.merge(Rm, on='date', how='outer')

df_merged = df_merged.rename(columns={'Rm': 'rm_US'})

df_merged.head()

,fic_AUSTRALIA,date,currency_AUSTRALIA,mportret_AUSTRALIA,mportretx_AUSTRALIA,fic_SWITZERLAND,currency_SWITZERLAND,mportret_SWITZERLAND,mportretx_SWITZERLAND,fic_GERMANY,currency_GERMANY,mportret_GERMANY,mportretx_GERMANY,fic_FRANCE,currency_FRANCE,mportret_FRANCE,mportretx_FRANCE,fic_UNITED KINGDOM,currency_UNITED KINGDOM,mportret_UNITED KINGDOM,mportretx_UNITED KINGDOM,fic_JAPAN,currency_JAPAN,mportret_JAPAN,mportretx_JAPAN,rm_US
0,AUS,2002-01-31,AUD,0.027358,0.027313,CHE,CHF,-0.022126,-0.022146,DEU,EUR,0.012277,0.011491,FRA,EUR,-0.027105,-0.027306,GBR,GBP,-0.010866,-0.011720,JPN,JPY,-0.056655,-0.056710,-0.015966
1,AUS,2002-02-28,AUD,-0.001832,-0.004358,CHE,CHF,0.011661,0.011646,DEU,EUR,-0.008807,-0.008816,FRA,EUR,-0.004394,-0.004447,GBR,GBP,-0.003341,-0.007072,JPN,JPY,0.042256,0.041998,-0.0217
2,AUS,2002-03-28,AUD,0.005199,-0.001914,CHE,CHF,0.048759,0.045570,DEU,EUR,0.049328,0.048470,FRA,EUR,0.052081,0.051943,GBR,GBP,0.038930,0.033661,JPN,JPY,0.053119,0.048294,0.044698
3,AUS,2002-04-30,AUD,-0.014302,-0.015424,CHE,CHF,-0.004109,-0.007961,DEU,EUR,-0.045031,-0.047093,FRA,EUR,-0.043988,-0.045755,GBR,GBP,-0.013362,-0.015411,JPN,JPY,0.020586,0.020574,-0.0496
4,AUS,2002-05-31,AUD,-0.000204,-0.001752,CHE,CHF,0.002963,-0.000639,DEU,EUR,-0.025703,-0.033068,FRA,EUR,-0.028016,-0.035487,GBR,GBP,-0.011482,-0.013937,JPN,JPY,0.035987,0.035920,-0.01051


In [38]:
Rf['date'] = Rm['date'].values

df_merged = df_merged.merge(Rf, on='date', how='outer')
df_merged = df_merged.rename(columns={'rf': 'rf_US'})

df_merged.head()

,fic_AUSTRALIA,date,currency_AUSTRALIA,mportret_AUSTRALIA,mportretx_AUSTRALIA,fic_SWITZERLAND,currency_SWITZERLAND,mportret_SWITZERLAND,mportretx_SWITZERLAND,fic_GERMANY,currency_GERMANY,mportret_GERMANY,mportretx_GERMANY,fic_FRANCE,currency_FRANCE,mportret_FRANCE,mportretx_FRANCE,fic_UNITED KINGDOM,currency_UNITED KINGDOM,mportret_UNITED KINGDOM,mportretx_UNITED KINGDOM,fic_JAPAN,currency_JAPAN,mportret_JAPAN,mportretx_JAPAN,rm_US,rf_US
0,AUS,2002-01-31,AUD,0.027358,0.027313,CHE,CHF,-0.022126,-0.022146,DEU,EUR,0.012277,0.011491,FRA,EUR,-0.027105,-0.027306,GBR,GBP,-0.010866,-0.011720,JPN,JPY,-0.056655,-0.056710,-0.015966,0.001408
1,AUS,2002-02-28,AUD,-0.001832,-0.004358,CHE,CHF,0.011661,0.011646,DEU,EUR,-0.008807,-0.008816,FRA,EUR,-0.004394,-0.004447,GBR,GBP,-0.003341,-0.007072,JPN,JPY,0.042256,0.041998,-0.0217,0.001444
2,AUS,2002-03-28,AUD,0.005199,-0.001914,CHE,CHF,0.048759,0.045570,DEU,EUR,0.049328,0.048470,FRA,EUR,0.052081,0.051943,GBR,GBP,0.038930,0.033661,JPN,JPY,0.053119,0.048294,0.044698,0.001425
3,AUS,2002-04-30,AUD,-0.014302,-0.015424,CHE,CHF,-0.004109,-0.007961,DEU,EUR,-0.045031,-0.047093,FRA,EUR,-0.043988,-0.045755,GBR,GBP,-0.013362,-0.015411,JPN,JPY,0.020586,0.020574,-0.0496,0.001461
4,AUS,2002-05-31,AUD,-0.000204,-0.001752,CHE,CHF,0.002963,-0.000639,DEU,EUR,-0.025703,-0.033068,FRA,EUR,-0.028016,-0.035487,GBR,GBP,-0.011482,-0.013937,JPN,JPY,0.035987,0.035920,-0.01051,0.001408


In [39]:
df_merged.to_csv('./data/final/merged_markets_no_fx_no_interbank.csv', index=False)

## Now adding fx and interbank to it

In [40]:
df_merged['date']

0     2002-01-31
1     2002-02-28
2     2002-03-28
3     2002-04-30
4     2002-05-31
         ...    
271   2024-08-30
272   2024-09-30
273   2024-10-31
274   2024-11-29
275   2024-12-31
Name: date, Length: 276, dtype: datetime64[ns]

In [41]:
import glob

paths_fx = glob.glob('data/fx/*.csv')

for path in paths_fx:
    curr = path.split('/')[-1].split('.')[0]
    print(f'the currency :{curr}')
    
    df_tamp = pd.read_csv(path)
    print(df_tamp.head())
    print('--'*20)

the currency :USD_GBP
  observation_date  EXUSUK
0       2002-01-01  1.4322
1       2002-02-01  1.4227
2       2002-03-01  1.4230
3       2002-04-01  1.4429
4       2002-05-01  1.4598
----------------------------------------
the currency :CHF_USD
  observation_date  EXSZUS
0       2002-01-01  1.6709
1       2002-02-01  1.6970
2       2002-03-01  1.6743
3       2002-04-01  1.6542
4       2002-05-01  1.5889
----------------------------------------
the currency :JPY_USD
  observation_date    EXJPUS
0       2002-01-01  132.6833
1       2002-02-01  133.6426
2       2002-03-01  131.0610
3       2002-04-01  130.7718
4       2002-05-01  126.3750
----------------------------------------
the currency :USD_AUD
  observation_date  EXUSAL
0       2002-01-01  0.5170
1       2002-02-01  0.5128
2       2002-03-01  0.5256
3       2002-04-01  0.5352
4       2002-05-01  0.5498
----------------------------------------
the currency :USD_EUR
  observation_date  EXUSEU
0       2002-01-01  0.8832
1       2002

In [45]:
df_tamp.head()

,observation_date,EXUSEU
0,2002-01-01,0.8832
1,2002-02-01,0.8707
2,2002-03-01,0.8766
3,2002-04-01,0.8860
4,2002-05-01,0.9170


In [55]:
fx_dfs = []

for path in paths_fx:

    print(f'Processing: {path}')
    
    df_fx = pd.read_csv(path)
    
    print(f'Number of rows: {len(df_fx)}')
    
    # df_country['date'] = Rm['date'].values
    
    # Rename columns to include country suffix, except 'date'
    df_fx = df_fx.rename(columns={col: f'fx_{col[2:]}' for col in df_fx.columns if col != 'observation_date'})
    
    # Append to list
    fx_dfs.append(df_fx)

# Merge all country-specific DataFrames on 'date'
from functools import reduce
df_merged_fx = reduce(lambda left, right: pd.merge(left, right, on='observation_date', how='outer'), fx_dfs)

# Optional: sort by date
df_merged_fx = df_merged_fx.sort_values(by='observation_date')

# Display the result
print(df_merged_fx.head())


Processing: data/fx/USD_GBP.csv
Number of rows: 276
Processing: data/fx/CHF_USD.csv
Number of rows: 276
Processing: data/fx/JPY_USD.csv
Number of rows: 276
Processing: data/fx/USD_AUD.csv
Number of rows: 276
Processing: data/fx/USD_EUR.csv
Number of rows: 276
  observation_date  fx_USUK  fx_SZUS   fx_JPUS  fx_USAL  fx_USEU
0       2002-01-01   1.4322   1.6709  132.6833   0.5170   0.8832
1       2002-02-01   1.4227   1.6970  133.6426   0.5128   0.8707
2       2002-03-01   1.4230   1.6743  131.0610   0.5256   0.8766
3       2002-04-01   1.4429   1.6542  130.7718   0.5352   0.8860
4       2002-05-01   1.4598   1.5889  126.3750   0.5498   0.9170


In [57]:
df_merged_fx = df_merged_fx.rename(columns={'observation_date': 'date'})

df_merged_fx.tail()

,date,fx_USUK,fx_SZUS,fx_JPUS,fx_USAL,fx_USEU
271,2024-08-01,1.2945,0.8579,146.2641,0.6658,1.1020
272,2024-09-01,1.3219,0.8472,142.9540,0.6770,1.1104
273,2024-10-01,1.3043,0.8613,149.8909,0.6700,1.0895
274,2024-11-01,1.2738,0.8809,153.7126,0.6528,1.0621
275,2024-12-01,1.2647,0.8916,153.8143,0.6334,1.0472


In [69]:
inter_dfs = []

for path in paths_rates:

    print(f'Processing: {path}')
    
    df_inter = pd.read_csv(path)
    
    print(f'Number of rows: {len(df_inter)}')
    
    # df_country['date'] = Rm['date'].values
    
    # Rename columns to include country suffix, except 'date'
    df_inter = df_inter.rename(columns={col: mapping_interbank[col] for col in df_inter.columns if col != 'observation_date'})
    
    # Append to list
    inter_dfs.append(df_inter)

# Merge all country-specific DataFrames on 'date'
from functools import reduce
df_merged_inter = reduce(lambda left, right: pd.merge(left, right, on='observation_date', how='outer'), inter_dfs)

# Optional: sort by date
df_merged_inter = df_merged_inter.sort_values(by='observation_date')
df_merged_inter.rename(columns={'observation_date': 'date'}, inplace=True)

# Display the result
print(df_merged_inter.head())


Processing: data/interbank_rates/euro_interbank.csv
Number of rows: 276
Processing: data/interbank_rates/ch_interbank.csv
Number of rows: 276
Processing: data/interbank_rates/us_interbank.csv
Number of rows: 276
Processing: data/interbank_rates/australia_interbank.csv
Number of rows: 276
Processing: data/interbank_rates/japan_interbank.csv
Number of rows: 273
Processing: data/interbank_rates/uk_interbank.csv
Number of rows: 276
         date  euro_interbank  ch_interbank  us_interbank  \
0  2002-01-01        3.338773      1.603725          1.74   
1  2002-02-01        3.357100      1.566671          1.82   
2  2002-03-01        3.390800      1.604164          1.91   
3  2002-04-01        3.406905      1.460647          1.87   
4  2002-05-01        3.467136      1.163494          1.82   

   australia_interbank  japan_interbank  uk_interbank  
0                 4.26              NaN       4.04677  
1                 4.31              NaN       4.05268  
2                 4.46           

In [70]:
df_merged_inter.head()

,date,euro_interbank,ch_interbank,us_interbank,australia_interbank,japan_interbank,uk_interbank
0,2002-01-01,3.338773,1.603725,1.74,4.26,NaN,4.04677
1,2002-02-01,3.357100,1.566671,1.82,4.31,NaN,4.05268
2,2002-03-01,3.390800,1.604164,1.91,4.46,NaN,4.13055
3,2002-04-01,3.406905,1.460647,1.87,4.59,0.10,4.17319
4,2002-05-01,3.467136,1.163494,1.82,4.84,0.08,4.15370


In [72]:
df_merged_fx_inter = pd.merge(df_merged_fx, df_merged_inter, on='date', how='outer')

df_merged_fx_inter.head()

,date,fx_USUK,fx_SZUS,fx_JPUS,fx_USAL,fx_USEU,euro_interbank,ch_interbank,us_interbank,australia_interbank,japan_interbank,uk_interbank
0,2002-01-01,1.4322,1.6709,132.6833,0.5170,0.8832,3.338773,1.603725,1.74,4.26,NaN,4.04677
1,2002-02-01,1.4227,1.6970,133.6426,0.5128,0.8707,3.357100,1.566671,1.82,4.31,NaN,4.05268
2,2002-03-01,1.4230,1.6743,131.0610,0.5256,0.8766,3.390800,1.604164,1.91,4.46,NaN,4.13055
3,2002-04-01,1.4429,1.6542,130.7718,0.5352,0.8860,3.406905,1.460647,1.87,4.59,0.10,4.17319
4,2002-05-01,1.4598,1.5889,126.3750,0.5498,0.9170,3.467136,1.163494,1.82,4.84,0.08,4.15370


In [73]:
df_merged_fx_inter.to_csv('./data/final/merged_fx_interbank_no_markets.csv', index=False)

In [74]:
df_merged_fx_inter['date'] = Rm['date'].values

df_merged_all = pd.merge(df_merged, df_merged_fx_inter, on='date', how='outer')

df_merged_all.head()

,fic_AUSTRALIA,date,currency_AUSTRALIA,mportret_AUSTRALIA,mportretx_AUSTRALIA,fic_SWITZERLAND,currency_SWITZERLAND,mportret_SWITZERLAND,mportretx_SWITZERLAND,fic_GERMANY,currency_GERMANY,mportret_GERMANY,mportretx_GERMANY,fic_FRANCE,currency_FRANCE,mportret_FRANCE,mportretx_FRANCE,fic_UNITED KINGDOM,currency_UNITED KINGDOM,mportret_UNITED KINGDOM,mportretx_UNITED KINGDOM,fic_JAPAN,currency_JAPAN,mportret_JAPAN,mportretx_JAPAN,rm_US,rf_US,fx_USUK,fx_SZUS,fx_JPUS,fx_USAL,fx_USEU,euro_interbank,ch_interbank,us_interbank,australia_interbank,japan_interbank,uk_interbank
0,AUS,2002-01-31,AUD,0.027358,0.027313,CHE,CHF,-0.022126,-0.022146,DEU,EUR,0.012277,0.011491,FRA,EUR,-0.027105,-0.027306,GBR,GBP,-0.010866,-0.011720,JPN,JPY,-0.056655,-0.056710,-0.015966,0.001408,1.4322,1.6709,132.6833,0.5170,0.8832,3.338773,1.603725,1.74,4.26,NaN,4.04677
1,AUS,2002-02-28,AUD,-0.001832,-0.004358,CHE,CHF,0.011661,0.011646,DEU,EUR,-0.008807,-0.008816,FRA,EUR,-0.004394,-0.004447,GBR,GBP,-0.003341,-0.007072,JPN,JPY,0.042256,0.041998,-0.0217,0.001444,1.4227,1.6970,133.6426,0.5128,0.8707,3.357100,1.566671,1.82,4.31,NaN,4.05268
2,AUS,2002-03-28,AUD,0.005199,-0.001914,CHE,CHF,0.048759,0.045570,DEU,EUR,0.049328,0.048470,FRA,EUR,0.052081,0.051943,GBR,GBP,0.038930,0.033661,JPN,JPY,0.053119,0.048294,0.044698,0.001425,1.4230,1.6743,131.0610,0.5256,0.8766,3.390800,1.604164,1.91,4.46,NaN,4.13055
3,AUS,2002-04-30,AUD,-0.014302,-0.015424,CHE,CHF,-0.004109,-0.007961,DEU,EUR,-0.045031,-0.047093,FRA,EUR,-0.043988,-0.045755,GBR,GBP,-0.013362,-0.015411,JPN,JPY,0.020586,0.020574,-0.0496,0.001461,1.4429,1.6542,130.7718,0.5352,0.8860,3.406905,1.460647,1.87,4.59,0.10,4.17319
4,AUS,2002-05-31,AUD,-0.000204,-0.001752,CHE,CHF,0.002963,-0.000639,DEU,EUR,-0.025703,-0.033068,FRA,EUR,-0.028016,-0.035487,GBR,GBP,-0.011482,-0.013937,JPN,JPY,0.035987,0.035920,-0.01051,0.001408,1.4598,1.5889,126.3750,0.5498,0.9170,3.467136,1.163494,1.82,4.84,0.08,4.15370


In [75]:
# Reorder columns: 'date' first, then 'fic_AUSTRALIA', then the rest
cols = df_merged_all.columns.tolist()

# Move 'date' and 'fic_AUSTRALIA' to the front
new_order = ['date', 'fic_AUSTRALIA'] + [col for col in cols if col not in ['date', 'fic_AUSTRALIA']]

# Apply the new order
df_merged_all = df_merged_all[new_order]

# Display to verify
df_merged_all.head()


,date,fic_AUSTRALIA,currency_AUSTRALIA,mportret_AUSTRALIA,mportretx_AUSTRALIA,fic_SWITZERLAND,currency_SWITZERLAND,mportret_SWITZERLAND,mportretx_SWITZERLAND,fic_GERMANY,currency_GERMANY,mportret_GERMANY,mportretx_GERMANY,fic_FRANCE,currency_FRANCE,mportret_FRANCE,mportretx_FRANCE,fic_UNITED KINGDOM,currency_UNITED KINGDOM,mportret_UNITED KINGDOM,mportretx_UNITED KINGDOM,fic_JAPAN,currency_JAPAN,mportret_JAPAN,mportretx_JAPAN,rm_US,rf_US,fx_USUK,fx_SZUS,fx_JPUS,fx_USAL,fx_USEU,euro_interbank,ch_interbank,us_interbank,australia_interbank,japan_interbank,uk_interbank
0,2002-01-31,AUS,AUD,0.027358,0.027313,CHE,CHF,-0.022126,-0.022146,DEU,EUR,0.012277,0.011491,FRA,EUR,-0.027105,-0.027306,GBR,GBP,-0.010866,-0.011720,JPN,JPY,-0.056655,-0.056710,-0.015966,0.001408,1.4322,1.6709,132.6833,0.5170,0.8832,3.338773,1.603725,1.74,4.26,NaN,4.04677
1,2002-02-28,AUS,AUD,-0.001832,-0.004358,CHE,CHF,0.011661,0.011646,DEU,EUR,-0.008807,-0.008816,FRA,EUR,-0.004394,-0.004447,GBR,GBP,-0.003341,-0.007072,JPN,JPY,0.042256,0.041998,-0.0217,0.001444,1.4227,1.6970,133.6426,0.5128,0.8707,3.357100,1.566671,1.82,4.31,NaN,4.05268
2,2002-03-28,AUS,AUD,0.005199,-0.001914,CHE,CHF,0.048759,0.045570,DEU,EUR,0.049328,0.048470,FRA,EUR,0.052081,0.051943,GBR,GBP,0.038930,0.033661,JPN,JPY,0.053119,0.048294,0.044698,0.001425,1.4230,1.6743,131.0610,0.5256,0.8766,3.390800,1.604164,1.91,4.46,NaN,4.13055
3,2002-04-30,AUS,AUD,-0.014302,-0.015424,CHE,CHF,-0.004109,-0.007961,DEU,EUR,-0.045031,-0.047093,FRA,EUR,-0.043988,-0.045755,GBR,GBP,-0.013362,-0.015411,JPN,JPY,0.020586,0.020574,-0.0496,0.001461,1.4429,1.6542,130.7718,0.5352,0.8860,3.406905,1.460647,1.87,4.59,0.10,4.17319
4,2002-05-31,AUS,AUD,-0.000204,-0.001752,CHE,CHF,0.002963,-0.000639,DEU,EUR,-0.025703,-0.033068,FRA,EUR,-0.028016,-0.035487,GBR,GBP,-0.011482,-0.013937,JPN,JPY,0.035987,0.035920,-0.01051,0.001408,1.4598,1.5889,126.3750,0.5498,0.9170,3.467136,1.163494,1.82,4.84,0.08,4.15370


In [76]:
df_merged_all.to_csv('./data/final/merged_all_data.csv', index=False)